In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from pandas import DataFrame, read_csv
import json

import sys
sys.path.insert(0, '../ToolBox')

import CrossValidation, MLP, Helper, ZScore, RandomForest

import os, sys
import cPickle

In [2]:
saved_train_data = open('pre_processed_data/train_preprocessed.pkl', 'rb')
normalized_train_data = cPickle.load(saved_train_data)

In [3]:
saved_test_data = open('pre_processed_data/test_preprocessed.pkl', 'rb')
normalized_test_data = cPickle.load(saved_test_data)

In [4]:
import yaml

with open("../ToolBox/params.json") as data_file:
    params = yaml.safe_load(data_file)

mlp = MLP.init(params['MLPandRF']['MLP'])
mlp['train'](normalized_train_data)

rf = RandomForest.init(params['MLPandRF']['RF'])
model = rf['train'](normalized_train_data)
result_dict = rf['predict'](normalized_test_data, model)
result_dict

Starting training.
Training.
Training finished.
Prediction:


array([ 5.,  1.,  5., ...,  5.,  4.,  4.], dtype=float32)

In [5]:
#lis = []
#lis.append("ID,Adoption,Died,Euthanasia,Return_to_owner,Transfer")

#for idx, z in enumerate(result_dict["network_0"]):
#    zList = z[0].tolist()
#    temp_str = str(idx+1)
#    for idy, x in enumerate(zList):
#        temp_str = temp_str+", "+str(x)
#    lis.append(temp_str)


#lis = np.array(lis)
#np.savetxt('result_shelter_animal_outcomes.csv', lis, fmt='%s', delimiter=',')